In [140]:
import requests
import pandas as pd
import folium

In [11]:
# ref : https://github.com/infoBMKG/data-gempabumi

autogempa = "https://data.bmkg.go.id/DataMKG/TEWS/autogempa.json"
gempabumiterkini = "https://data.bmkg.go.id/DataMKG/TEWS/gempaterkini.json"
gempabumidirasakan = "https://data.bmkg.go.id/DataMKG/TEWS/gempadirasakan.json"

# Function

In [132]:
class StoreBMKG():
    def __init__(self, data: str):
        self.__get_bmkg(data)

    def __auto_earthquake(self, filename, data: dict):
        data = pd.DataFrame.from_dict(data, orient='index').T
        data.to_csv('dataset/'+filename+'.csv', index=True)

    # need to improve table for more tidy csv from felt to new
    def __felt_earthquake(self, filename, data: pd.DataFrame):
        columns = ['Tanggal','Jam','DateTime','Coordinates','Lintang','Bujur','Magnitude','Kedalaman','Wilayah', 'Dirasakan']

        rows = []
        for d in data:
            rows.append(pd.Series(d, index=columns))
        
        datas = pd.DataFrame(rows, columns=columns)
        datas.to_csv('dataset/'+filename+'.csv', index=True)
            
    def __new_earthquake(self, filename, data: pd.DataFrame):
        columns = ['Tanggal','Jam','DateTime','Coordinates','Lintang','Bujur','Magnitude','Kedalaman','Wilayah', 'Potensi']

        rows = []
        for d in data:
            rows.append(pd.Series(d, index=columns))
        
        datas = pd.DataFrame(rows, columns=columns)
        datas.to_csv('dataset/'+filename+'.csv', index=True)

    def __get_bmkg(self, data:str):

        from io import StringIO
        
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 \
                    (Macintosh; Intel Mac OS X 10_11_5) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/50.0.2661.102 Safari/537.36'
            }
            
            response = requests.get(data, headers=headers)
            raw = pd.read_json(StringIO(response.content.decode('utf-8')))
            
            filename = data.split('/')[-1].split('.')[0]

            if filename == 'autogempa':
                self.__auto_earthquake(filename, raw['Infogempa']['gempa'])
            elif filename == 'gempadirasakan':
                self.__felt_earthquake(filename, raw['Infogempa']['gempa'])
            elif filename == 'gempaterkini':
                self.__new_earthquake(filename, raw['Infogempa']['gempa'])
                
        except requests.exceptions.HTTPError as errh:
            print(repr(errh))
        except requests.exceptions.ConnectionError as errc:
            print(repr(errc))
        except requests.exceptions.Timeout as errt:
            print(repr(errt))
        except requests.exceptions.RequestException as err:
            print(repr(err))

# Storing Data into CSV

In [137]:
listDataBMKG = [autogempa, gempabumidirasakan, gempabumiterkini]
for d in listDataBMKG:
    StoreBMKG(d)

# Data BMKG Gempa

## Show Table

In [139]:
dataAutoGempa = pd.read_csv('dataset/autogempa.csv')

dataAutoGempa.drop(dataAutoGempa.columns[
    dataAutoGempa.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

display(dataAutoGempa)

,Tanggal,Jam,DateTime,Coordinates,Lintang,Bujur,Magnitude,Kedalaman,Wilayah,Potensi,Dirasakan,Shakemap
0,26 Mar 2024,00:36:36 WIB,2024-03-25T17:36:36+00:00,"0.83,125.40",0.83 LU,125.40 BT,4.8,40 km,Pusat gempa berada di laut 71 km Tenggara Ratahan,Gempa ini dirasakan untuk diteruskan pada masy...,"II-III Bitung, II Manado, II Tondano",20240326003636.mmi.jpg


## Show Map

### Folium

In [170]:
quake = [0, 0]
surabaya = [-7.2754417, 112.6302827]

for v in dataAutoGempa['Coordinates'].values:
    quake = v.split(',')

centerlat = (float(quake[0]) + float(quake[1])) / 2
centerlon = (surabaya[0] + surabaya[1]) / 2
map = folium.Map(location=[centerlat, centerlon], zoom_start=7)

folium.Marker(quake).add_to(map)
folium.Marker(surabaya).add_to(map)

map


# Data BMKG Gempa Dirasakan

# Data BMKG Gempa Terkini

In [135]:
DataBMKG(gempabumiterkini)